In [ ]:
# load csv
import pandas as pd
csv_path = r"https://docs.google.com/spreadsheets/d/1EvIFLGpsKvuOgdX6F9MYlGl9rCLjbKHPlE8srBDMwYo/edit?usp=sharing"

# Download file from Google Sheets
url = csv_path.replace('/edit?usp=sharing', '/export?format=csv')
df = pd.read_csv(url)

df.columns


In [ ]:
import requests
item_csv_path = r"https://docs.google.com/spreadsheets/d/1eRkhdeDaD60IOajReYVRId28CM4IKQJiheL-yBWfzUw/edit?usp=sharing"
url = item_csv_path.replace('/edit?usp=sharing', '/export?format=xlsx')


item_df = pd.read_excel(url, sheet_name="Mac-Docs")

api_id = {}

for index, row in item_df.iterrows():
    value = (row["item id"], row["option id"])
    key = (row["item"], row["CPU"], row["CPU Core"], row["GPU Core"], row["RAM"], row["SSD"])
    api_id[key] = value

len(api_id)

In [ ]:
import datetime

start_date = datetime.datetime(2023, 3, 1)
end_date = start_date + datetime.timedelta(days=35)

# models
stats = {}
for week in range(4):
    for model in df["model"].unique().tolist():
        for screen_size in df[df["model"] == model]["screen_size"].unique():
            for chip in df[(df["model"] == model) & (df["screen_size"] == screen_size)]["chip"].unique():
                for core in df[(df["model"] == model) & (df["screen_size"] == screen_size) & (df["chip"] == chip)]["core"].unique():
                    for ram in df[(df["model"] == model) & (df["screen_size"] == screen_size) & (df["chip"] == chip) & (df["core"] == core)]["ram"].unique():
                        for ssd in df[(df["model"] == model) & (df["screen_size"] == screen_size) & (df["chip"] == chip) & (df["core"] == core) & (df["ram"] == ram)]["ssd"].unique():
                           for unopened in [True, False]:
                                entity_list = df[(df["model"] == model) & (df["screen_size"] == screen_size) & (df["chip"] == chip) & (df["core"] == core) & (df["ram"] == ram) & (df["ssd"] == ssd) & (df["unused"] == unopened)].sort_values(by="date").sort_values(by="price for statistics")
                                # 날짜 필터링
                                entity_list = entity_list[(entity_list["date"] >= start_date.strftime("%Y-%m-%d")) & (entity_list["date"] <= end_date.strftime("%Y-%m-%d"))]

                                print(f"{start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}, {model}, {screen_size}, {chip}, {core}, {ram}, {ssd}, {'미개봉' if unopened else '중고', len(entity_list)}")
                                if entity_list.size <= 1:
                                    continue

                                # 전처리
                                entity_list["price for statistics"] = entity_list["price for statistics"].apply(lambda x: int(x.replace("원", "").replace(",", "")))

                                # 평균
                                mean = entity_list["price for statistics"].mean(numeric_only=True)

                                # 중앙 값
                                median = entity_list["price for statistics"].median()

                                # 표준 편차
                                std = entity_list["price for statistics"].std()

                                # 평균으로부터 표준편차 이내의 값 평균
                                mean_std = entity_list[(entity_list["price for statistics"] >= (mean - std)) & (entity_list["price for statistics"] <= (mean + std))]["price for statistics"].mean()

                                # 상위 0 ~ 20% 사이 값들의 평균
                                top_20_percentile = entity_list["price for statistics"].quantile(0.8)
                                top_20_percent_values = entity_list[entity_list["price for statistics"] >= top_20_percentile]
                                mean_20 = top_20_percent_values.mean(numeric_only=True)["price for statistics"]

                                # 하위 20% 평균
                                bottom_20_percentile = entity_list["price for statistics"].quantile(0.2)
                                bottom_20_percent_values = entity_list[entity_list["price for statistics"] <= bottom_20_percentile]
                                mean_80 = bottom_20_percent_values.mean(numeric_only=True)["price for statistics"]


                                # id 만들기
                                if model == "Macbook Pro":
                                    _model = f"{model} {screen_size}"
                                else:
                                    _model = model

                                _cpu = chip.replace(" ", "")
                                _cpu_core, _gpu_core = map(lambda x: int(x), core.replace(" ", "").split(","))
                                _ram = int(ram.replace("GB", "").replace(" ", ""))
                                _ssd = ssd.replace(" ", "")

                                key = (_model, chip, _cpu_core, _gpu_core, _ram, _ssd)
                                if key not in api_id:
                                    print(f"key not found: {key}")
                                    continue
                                else:
                                    item_id, option_id = api_id[key]
                                    # print(f"item_id: {item_id}, option_id: {option_id}")

                                #
                                # print(f" - 평균 : {round(mean, 1)}원\n"
                                #       f" - 중앙 값 : {round(median, 1)}원\n"
                                #       f" - 표준 편차 : {round(std, 1)}원\n"
                                #       f" - 평균으로부터 표준편차 이내의 값 평균 : {round(mean_std, 1)}원\n"
                                #       f" - 상위 20% 평균 : {round(mean_20, 1)}원\n"
                                #       f" - 하위 20% 평균 : {round(mean_80, 1)}원\n")
                                #
                                stats[(item_id, option_id, start_date.strftime("%Y-%m-%d"), unopened)] = {
                                    "start_date": start_date.strftime("%Y-%m-%d"),
                                    "end_date": end_date.strftime("%Y-%m-%d"),
                                    "mean": round(mean, 1),
                                    "median": round(median, 1),
                                    "std": round(std, 1),
                                    "mean_std": round(mean_std, 1),
                                    "mean_20": round(mean_20, 1),
                                    "mean_80": round(mean_80, 1),
                                }

    start_date += datetime.timedelta(days=7)
    end_date += datetime.timedelta(days=7)


In [ ]:

outputs = []
for item_id in range(1, 100):
    for option_id in range(1, 100):
        for unopened in [True, False]:
            if (item_id, option_id) not in api_id.values():
                continue

            start_date = datetime.datetime(2023, 3, 1)
            end_date = start_date + datetime.timedelta(days=35)

            for week in range(4):
                csv = {
                    "start_date": start_date.strftime("%Y-%m-%d"),
                    "end_date": end_date.strftime("%Y-%m-%d"),
                    "item_id": item_id,
                    "option_id": option_id,
                    "mean_std": 0,
                    "mean_20": 0,
                    "mean_80": 0,
                    "unopend": unopened
                }

                if (item_id, option_id, start_date.strftime("%Y-%m-%d"), unopened) in stats:
                    values = stats[(item_id, option_id, start_date.strftime("%Y-%m-%d"), unopened)]
                    if values["mean_20"] == values["mean_80"]:
                        values["mean_std"] = values["mean_20"]
                        values["mean_20"] = 0
                        values["mean_80"] = 0

                    csv["start_date"] = values["start_date"]
                    csv["end_date"] = values["end_date"]
                    csv["mean_std"] = values["mean_std"]
                    csv["mean_20"] = values["mean_20"]
                    csv["mean_80"] = values["mean_80"]

                outputs.append(csv)
                start_date += datetime.timedelta(days=7)
                end_date += datetime.timedelta(days=7)

df = pd.DataFrame(outputs)
df.to_csv("stats.csv", index=False)

In [187]:
import time
# API Call
api_url_template = "https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/{}/option/{}"

# read csv
df = pd.read_csv("stats.csv")

for index, row in df.iterrows():
    item_id = row["item_id"]
    option_id = row["option_id"]
    date = row["end_date"]
    mean = row["mean_std"]
    high = row["mean_20"]
    low = row["mean_80"]
    unopend = row["unopend"]


    api_url = api_url_template.format(item_id, option_id)
    if unopend:
        api_url += "?unopened=true"
    else:
        api_url += "?unopened=false"

    print(api_url)

    data = {
        "date": date,
        "mid": int(mean),
        "high": int(high),
        "low": int(low),
    }


    print(f"{data}")
    # response = requests.post(api_url, json=data)
    # print(response.status_code)
    # print(response.text)
    # print()
    # time.sleep(1)


https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-05', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-12', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-19', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-26', 'mid': 0, 'high': 0, 'low': 0}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=false
{'date': '2023-04-05', 'mid': 540000, 'high': 700000, 'low': 530000}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=false
{'date': '2023-04-12', 'mid': 576000, 'high': 685000, 'low': 530000}
https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopene